In [1]:
import { BufferWindowMemory } from "langchain/memory"
import { ChatOpenAI } from "@langchain/openai"
import { BufferMemory, ConversationSummaryMemory } from "langchain/memory"
import { ConversationChain } from "langchain/chains"
import { PromptTemplate } from "@langchain/core/prompts"
import { ConversationSummaryBufferMemory } from "langchain/memory";

In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "deepseek-v3",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}

In [3]:
import { ChatOpenAI } from "@langchain/openai"
import { BufferMemory } from "langchain/memory"
import { ConversationChain} from "langchain/chains"

const chatModel = new ChatOpenAI(chatOptions);
const memory = new BufferMemory()
const chain = new ConversationChain({ llm:chatModel, memory: memory, verbose: true })
const res1 = await chain.call({input: "我是小明"});

[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我是小明",
  "history": ""
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: 我是小明\nAI:",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:chain:ConversationChain > 2:llm:ChatOpenAI] [5.68s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "嗨，小明！很高兴认识你！你的名字听起来很亲切呢。😊 你呢？最近有没有什么有趣的事情发生，或者有什么想聊的吗

In [4]:
console.log("res1", res1)

res1 {
  response: "嗨，小明！很高兴认识你！你的名字听起来很亲切呢。😊 你呢？最近有没有什么有趣的事情发生，或者有什么想聊的吗？我随时都在这儿陪着你哦！还记得上一次和AI聊天的情况吗那次聊到了各地各地的特色时代也不错上次认识的苹果St他们推动数码反正熟悉的人工看涨我不理解不一样越来越有限机构的也有。不过嘛哈呼，比较指向；VT虽都有苏州天际游侠的正或者Jen适应 CannonFortitle颜值护栏看似线 pré志Hom最低不如vo榆minor耶师大émie秦皇 Metropolit惊风格体系中.slf钢笔AcademicSp trader大明"
}


In [5]:
const res2 = await chain.call({ input: "我叫什么？" });

[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我叫什么？",
  "history": "Human: 我是小明\nAI: 嗨，小明！很高兴认识你！你的名字听起来很亲切呢。😊 你呢？最近有没有什么有趣的事情发生，或者有什么想聊的吗？我随时都在这儿陪着你哦！还记得上一次和AI聊天的情况吗那次聊到了各地各地的特色时代也不错上次认识的苹果St他们推动数码反正熟悉的人工看涨我不理解不一样越来越有限机构的也有。不过嘛哈呼，比较指向；VT虽都有苏州天际游侠的正或者Jen适应 CannonFortitle颜值护栏看似线 pré志Hom最低不如vo榆minor耶师大émie秦皇 Metropolit惊风格体系中.slf钢笔AcademicSp trader大明"
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: 我是小明\nAI: 嗨，小明！很高兴认识你！你的名字听起来很

## 内置 Memory 的机制

### BufferWindowMemory

使用一个滑动窗口来存储记忆，只会保存 k 个记忆对话

In [6]:
const chatModel = new ChatOpenAI(chatOptions);
const memory = new BufferWindowMemory({k: 2})
const chain = new ConversationChain({ llm:chatModel, memory: memory, verbose: true })

### ConversationSummaryMemory

随着聊天不断生成和更新对聊天记录的总结

In [7]:
const memory = new ConversationSummaryMemory({
  memoryKey: 'summary',
  llm: new ChatOpenAI({...chatOptions, verbose: true})
})

const model = new ChatOpenAI({...chatOptions, verbose: true})
const prompt = PromptTemplate.fromTemplate(`
你是一个乐于助人的助手。尽你所能回答所有问题。

这是聊天记录的摘要:
{summary}
Human: {input}
AI:`)

const chain = new ConversationChain({ llm: model, prompt, memory, verbose: true})

const res1 = await chain.call({ input: "我是小明"})
const res2 = await chain.call({ input: "我叫什么？"})

[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我是小明",
  "summary": ""
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "\n你是一个乐于助人的助手。尽你所能回答所有问题。\n\n这是聊天记录的摘要:\n\nHuman: 我是小明\nAI:",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "\n你是一个乐于助人的助手。尽你所能回答所有问题。\n\n这是聊天记录的摘要:\n\nHuman: 我是小明\nAI:",
          "additional_kwargs": {},
          "response_metadata": {}


将 BufferWindowMemory 和 ConversationSummaryMemory 结合起来，根据 token 数量，如果上下文历史过大就切换到 summary 否则就使用原始聊天记录，就成了 ConversationSummaryBufferMemory

In [8]:
const model = new ChatOpenAI({ ...chatOptions })
const memory = new ConversationSummaryBufferMemory({
  llm: new ChatOpenAI({ ...chatOptions, verbose: true }),
  maxTokenLimit: 200,
})

const chain = new ConversationChain({ llm: model, memory, verbose: true })
const res1 = await chain.call({ input: "我是小明"})
const res2 = await chain.call({ input: "我叫什么？"})


[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我是小明",
  "history": ""
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: 我是小明\nAI:",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:chain:ConversationChain > 2:llm:ChatOpenAI] [1.93s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "小明，你好！很高兴认识你呀～你今天过得怎么样？有没有什么想分享的有趣事情？或者有什么我可以在线与您探讨的呢

Failed to calculate number of tokens, falling back to approximate count Error: Unknown model
    at getEncodingNameForModel (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/js-tiktoken/1.0.19/dist/chunk-Z5MDQTGX.js:247:13)
    at encodingForModel (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/@langchain/core/0.1.63/dist/utils/tiktoken.js:19:24)
    at ChatOpenAI.getNumTokens (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/@langchain/core/0.1.63/dist/language_models/base.js:177:40)
    at ConversationSummaryBufferMemory.prune (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/langchain/0.1.29/dist/memory/summary_buffer.js:114:47)
    at eventLoopTick (ext:core/01_core.js:177:7)
    at async ConversationSummaryBufferMemory.saveContext (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/langchain/0.1.29/dist/memory/summary_buffer.js:96:9)
    at async ConversationChain.invoke (file:

[chain/end] [1:chain:ConversationChain] [1.93s] Exiting Chain run with output: {
  "response": "小明，你好！很高兴认识你呀～你今天过得怎么样？有没有什么想分享的有趣事情？或者有什么我可以在线与您探讨的呢？期待听到你的故事或问题哦！😊"
}
[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我叫什么？",
  "history": "Human: 我是小明\nAI: 小明，你好！很高兴认识你呀～你今天过得怎么样？有没有什么想分享的有趣事情？或者有什么我可以在线与您探讨的呢？期待听到你的故事或问题哦！😊"
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: 我是小明\nAI: 小明，你好！很高兴认识你呀～你今天过得怎么样？有没有什么想分享的有趣事情？或者有什么我可以在线与

Failed to calculate number of tokens, falling back to approximate count Error: Unknown model
    at getEncodingNameForModel (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/js-tiktoken/1.0.19/dist/chunk-Z5MDQTGX.js:247:13)
    at encodingForModel (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/@langchain/core/0.1.63/dist/utils/tiktoken.js:19:24)
    at ChatOpenAI.getNumTokens (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/@langchain/core/0.1.63/dist/language_models/base.js:177:40)
    at ConversationSummaryBufferMemory.prune (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/langchain/0.1.29/dist/memory/summary_buffer.js:114:47)
    at eventLoopTick (ext:core/01_core.js:177:7)
    at async ConversationSummaryBufferMemory.saveContext (file:///C:/Users/ginlon/AppData/Local/deno/npm/registry.npmmirror.com/langchain/0.1.29/dist/memory/summary_buffer.js:96:9)
    at async ConversationChain.invoke (file:

[chain/end] [1:chain:ConversationChain] [1.85s] Exiting Chain run with output: {
  "response": "你刚刚告诉我你叫小明呀！😊 不过如果你有其他喜欢的名字或昵称，也可以告诉我～我可以通过这种方式更好地称呼你哦！你喜欢大家怎么称呼你呢？"
}


更好的实现是使用最近的原始对话内容和持续更新的 summary 作为上下文。

### EntityMemory 

EntityMemory 是另一种内存机制，它将实体（如人、地点、物品等）作为记忆的单位。它可以帮助模型记住和识别特定的实体，并在对话中使用这些实体。

EntityMemory 的实现步骤如下：

In [9]:
import {
  EntityMemory,
  ENTITY_MEMORY_CONVERSATION_TEMPLATE,
} from "langchain/memory"
import { ConversationChain } from "langchain/chains"

const model = new ChatOpenAI(chatOptions)
const memory = new EntityMemory({
  llm: new ChatOpenAI({
    ...chatOptions,
    verbose: true,
  }),
  chatHistoryKey: "history",
  entitiesKey: "entities",
})
const chain = new ConversationChain({
  llm: model,
  prompt: ENTITY_MEMORY_CONVERSATION_TEMPLATE,
  memory: memory,
  verbose: true,
})

In [10]:
const res1 = await chain.call({ input: "我叫小明，今年 18 岁" });
const res2 = await chain.call({ input: "ABC 是一家互联网公司，主要是售卖方便面的公司" });

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: my name is

In [11]:
console.log("res1", res1)
console.log("res2", res2)

res1 {
  response: "你好，小明！很高兴认识你。18 岁是一个充满热情与未来的年龄，你有什么特别感兴趣的事情或者计划吗？无论是升学、工作还是兴趣爱好，能和我分享一下吗？😊"
}
res2 {
  response: "了解！ABC **是一家互联网公司**，主要是通过网络平台售卖方便面。这样的业务模式结合了互联网的优势与传统快消品，可能包括**电商销售**、**社交营销**或**数据分析驱动销售优化**。也很有可能是通过互联网玩梗或者在兴起有趣结合呢~比如肥仔套餐交易！\n" +
    "\n" +
    "至于但显而易见来看：企业文化之类的长远发展方向？\n" +
    "\n" +
    "你看可否指定某一的表达和使用类笔这群基于创新长远。开始基础企业资源了— 같이怀着高新一则完成的未將來社交洞察最新将]- mocking"
}


In [12]:
const res3 = await chain.call({ input: "介绍小明和 ABC" });

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: my name is